# FLUORIMETRIA: spettro emissione

si hanno soluzioni di rodamina in acqua consequenzialmente a concentrazioni minori

OBIETTIVI:
viene variata la concentrazione di rodamina. Si vuole mettere in relazione l'assorbanza di rodamina con l'intensità di fluorescenza

$\epsilon$ = coefficiente estinzione molare rodamina 6G = $116000 M^{-1} cm^{-1} $ 

$A = \epsilon C l$

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit

Assorbanza:

Segue il calcolo delle assorbanze, in laboratorio è stato usato lo spettrofotometro per misurare l'assorbanza del campione iniziale. In seguito la soluzione è stata diluita usando $A_i V_i = A_f V_f$ (valida in quanto $C \propto A$)

In [2]:
λexc = 526 #nm frequenza di eccitazione
SIGMA_LAMBDA = 1.5 #nm DICHIARATI DAL COSTRUTTORE

# Extracting the relevant columns for plotting
WAVELENGHTS = ['λ 5', 'λ 4', 'λ 3', 'λ 2', 'λ 1', 'λ 0.8', 'λ 0.6', 'λ 0.4', 'λ 0.2', 'λ 0.1']
INTENSITIES = ['I 5', 'I 4', 'I 3', 'I 2', 'I 1', 'I 0.8', 'I 0.6', 'I 0.4', 'I 0.2', 'I 0.1']
# CONCENTRATIONS=[5,4,3,2,1,0.8,0.6,0.4,0.2,0.1]

# FUNZIone per estrarre i massimi con fit parabolico
def max_fit_parabolic(x, λcenter, a, IMAX): # -a*(x-λcenter)**2 + IMAX
    return -a*(x-λcenter)**2 + IMAX

In [3]:
# # real rodamina 
# R6G_emission = pd.read_csv(r'.\data\Rhodamine 6G_emission.csv', sep=',')
# R6G_absorbance = pd.read_csv(r'.\data\Rhodamine 6G_absorbance.csv', sep=',')
# R6G_excitation = pd.read_csv(r'.\data\Rhodamine 6G_excitation.csv', sep=',')

Assorbanza iniziale del campione di rodamina:

In [4]:
# plt.plot(ass_rodamina['λ'],ass_rodamina['A'])

# determinazione della concentrazione iniziale da spettro di assorbimento

filename = './data/rodamina_ass_05_g1.TXT'

ass_rodamina = pd.read_csv(filename,sep='\t',header=None,skiprows=19,nrows=208) # 630 - 423 nm
ass_rodamina.columns = ['λ','A']

filename = './data/fondo acqua.TXT'

fondo_acqua = pd.read_csv(filename,sep='\t',header=None,skiprows=19,nrows=251)
fondo_acqua.columns = ['λ','A']
fondo_acqua = fondo_acqua[(fondo_acqua['λ'] <= 630) & (fondo_acqua['λ'] >= 423)]

fondo_acqua.tail()

,λ,A
223,427.0,0.06933
224,426.0,0.07005
225,425.0,0.07090
226,424.0,0.07070
227,423.0,0.07054


In [5]:
fig = go.Figure()

# Add the measured absorbance spectrum
fig.add_trace(go.Scatter(
    x=ass_rodamina['λ'],
    y=ass_rodamina['A'],
    mode='lines',
    name='Signal',
    opacity=0.3
))

fig.add_trace(go.Scatter(
    x=fondo_acqua['λ'],
    y=fondo_acqua['A'],
    mode='lines',
    name='Background',
    opacity=0.3
))

fig.add_trace(go.Scatter(
    x=fondo_acqua['λ'],
    y=ass_rodamina['A'] - fondo_acqua['A'],
    mode='lines',
    name='Spectrum Rhodamine 6G',
    line=dict(width=4)  # Make the line thicker
))


# Update layout for the plot
fig.update_layout(
    # title='Absorbance Spectrum of Rodamine',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Absorbance',
    legend=dict(x=1, y=1, xanchor='right', yanchor='top'),  # Position legend at the top right
    font=dict(size=14),
    height=600,
    width=800
)

fig.write_html(dir_path +r"\html\absorbance_spectrum.html")
fig.write_image(dir_path + r"\images\absorbance_spectrum.png")
fig.show()

#### fit per determinare il massimo

In [6]:
λ_ass = ass_rodamina['λ'].to_numpy()
A_rod = ass_rodamina['A'].to_numpy() - fondo_acqua['A'].to_numpy()

l0 = ass_rodamina['A'].idxmax()

popt, pcov = curve_fit(max_fit_parabolic, λ_ass[l0-5:l0+5], A_rod[l0-5:l0+5], p0=[530, 2, 0.5])
λcenter, a, A = popt
error_λcenter, error_a, error_A = np.sqrt(np.diag(pcov))

print('Picco di assorbimento A: ',final_val(A,error_A,decimals=4))
print(A,error_A)

Picco di assorbimento A:  0.48 ± 0.0003 
0.4800416799849726 0.0003353431037236748


In [7]:
A0 = A
sigmaA0 = error_A
print('ASSORBANZA INIZIALE:')
print('a0:',final_val(A0,sigmaA0,decimals=4,udm='μM'))

ASSORBANZA INIZIALE:
a0: 0.48 ± 0.0003 μM


determinazione di tutte le concentrazioni di cui è stato fatto lo spettro di fluorescenza:

in laboratorio è stata raccolta la misura di:

- $V_{in}$ = volume estratto dalla soluzione precedente
- $V_{fin}$ = volume estratto dalla soluzione precedente + tampone (acqua)

In [8]:
from IPython.display import display_html

filename = './data/concentrazioni_rodamina.csv'

volumi = pd.read_csv(filename,sep=',')
volumi = volumi[['Vi (muL)','Vf (muL)']]
# Display the DataFrame horizontally
display(volumi.T)

,0,1,2,3,4,5,6,7,8
Vi (muL),2210,2252,2000,1511,2423,2251,2002,1497,1504
Vf (muL),3008,3015,3015,3016,3010,3002,3000,2994,3003


In [9]:
#Cf Vf = Ci Vi -> Cf = Ci Vi / Vf
# sigma V = 1 μL
ASS = [A0]
SIGMA_ASS = [sigmaA0]
sigmaV = 1 # μL, comodo perchè si ha sempre a che fare con rapporti
for idx, row in enumerate(volumi.iterrows()):
    vi, vf = row[1]['Vi (muL)'], row[1]['Vf (muL)']
    An = ASS[idx] * vi / vf
    sAn = An * np.sqrt((sigmaV/vi)**2 + (sigmaV/vf)**2 + (SIGMA_ASS[idx]/ASS[idx])**2) 
    ASS.append(An)
    SIGMA_ASS.append(sAn)

ASS = np.array(ASS)
SIGMA_ASS = np.array(SIGMA_ASS)

# Create a pandas DataFrame to display ASS and SIGMA_ASS
assorbance_df = pd.DataFrame({
    'A': ASS,
    'σA': SIGMA_ASS
})
display(assorbance_df.T)

,0,1,2,3,4,5,6,7,8,9
A,0.480042,0.352690,0.263436,0.174750,0.087549,0.070475,0.052845,0.035265,0.017633,0.008831
σA,0.000335,0.000316,0.000278,0.000212,0.000124,0.000107,0.000085,0.000061,0.000033,0.000018


ora che sono noti i valori delle concentrazioni dei vari campioni di cui è stata fatto lo spettro di fluorescenza si importano i veri e propri spettri di fluorescenza

In [10]:
filename = 'data/assorbanza_rodamina_concentrazioni_tot.csv'

header = ['λ 5','I 5','λ 4','I 4','no λ 4','no I 4','λ 3','I 3','λ 2','I 2','λ 1','I 1','λ 0.8','I 0.8','λ 0.6','I 0.6','λ 0.4','I 0.4','λ 0.2','I 0.2','λ 0.1','I 0.1']

# fondo = pd.read_csv('./data/fondoH20Sspettrofluorimetro.csv',sep=',',header=1,nrows=150)[['Wavelength (nm).1','Intensity (a.u.).1']]
# fondo.columns = ['λfondo','Ifondo']
# fondo.tail()

# from scipy.interpolate import interp1d
# fondo_interp = interp1d(fondo['λfondo'], fondo['Ifondo'])


data = pd.read_csv(filename,sep=',',header=1,nrows=117)
data = data.iloc[:, :-1] # drop last column
data.columns = header
data.tail()

,λ 5,I 5,λ 4,I 4,no λ 4,no I 4,λ 3,I 3,λ 2,I 2,...,λ 0.8,I 0.8,λ 0.6,I 0.6,λ 0.4,I 0.4,λ 0.2,I 0.2,λ 0.1,I 0.1
112,646.950012,29.311554,646.950012,25.126436,646.950012,29.019300,646.950012,21.187140,646.950012,14.265793,...,646.950012,4.883633,646.950012,3.315377,646.950012,1.589167,646.950012,0.555993,646.950012,0.351009
113,647.969971,28.124762,647.969971,23.946281,647.969971,27.984386,647.969971,20.061258,647.969971,13.714191,...,647.969971,4.719029,647.969971,3.173567,647.969971,1.429973,647.969971,0.563877,647.969971,0.378182
114,648.840027,26.902561,648.840027,23.194933,648.840027,27.167797,648.840027,19.790516,648.840027,13.449127,...,648.840027,4.602462,648.840027,3.138167,648.840027,1.417011,648.840027,0.545661,648.840027,0.333807
115,649.849976,26.203035,649.849976,22.347122,649.849976,25.561733,649.849976,18.675690,649.849976,12.785388,...,649.849976,4.334819,649.849976,2.955356,649.849976,1.345403,649.849976,0.519533,649.849976,0.366366
116,650.859985,24.811270,650.859985,21.325739,650.859985,24.539951,650.859985,18.069601,650.859985,12.159359,...,650.859985,4.134296,650.859985,2.907012,650.859985,1.283329,650.859985,0.497435,650.859985,0.298861


plot spettri di fluo

In [11]:
fig = go.Figure()

for λ_col, I_col, a in zip(WAVELENGHTS, INTENSITIES, ASS):
    fig.add_trace(go.Scatter(x=data[λ_col], y=data[I_col],
                    mode='lines',
                    name=f'{np.round(a, 2)}'))
    
# fig.add_trace(go.Scatter(x=fondo['λfondo'],y=fondo['Ifondo'],
#                     mode='lines',
#                     name=f'fondo'))

fig.update_layout(
                  xaxis_title='Wavelength (nm)',
                  yaxis_title='Intensity (a.u.)',
                  height=600,
                  width=800,
                  yaxis=dict(range=[0, 1000]),
                  legend=dict(x=0.9, y=0.9),
                  font=dict(size=14))

fig.write_html(dir_path + r"\html\emission_spectrum.html")
fig.write_image(dir_path + r"\images\emission_spectrum.png")
fig.show()

segue fit parabolico dei picchi dello spettro, vengono considerati solo i 5 punti a destra e 5 a sinistra del punto massimo dello spettro

segue l'estrapolazione del valore di Imax

In [12]:
λcenter_list = []
a_list = []
IMAX_list = []
error_λcenter_list = []
error_a_list = []
error_IMAX_list = []
sigmay = [] # contiene l'errore propagato con x
for λ_col, I_col in zip(WAVELENGHTS, INTENSITIES):
    Λ = data[λ_col]
    I = data[I_col]
    Λmaxcenter = I.idxmax()

    λcenter = Λ[Λmaxcenter] # guess
    IMAX = I.max() # guess
    a = 1 # guess

    xrangemax = Λ[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max
    yrangemax = I[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max
    # ---- per considerare errore su lambda di 1.5nm
    # iterazione 0
    popt, pcov = curve_fit(max_fit_parabolic, xrangemax, yrangemax, p0=[λcenter, a, IMAX])
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))
    # iterazione 1
    dydl = np.abs(-2*a*(λcenter-xrangemax)) # derivata prima rispetto a λ
    sy = dydl*SIGMA_LAMBDA
    popt, pcov = curve_fit(max_fit_parabolic, xrangemax, yrangemax, p0=[λcenter, a, IMAX],sigma=sy)
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))
    # iterazione 2
    d2ydl2 = np.abs(-2*a) # derivata seconda rispetto a λ
    sy = np.sqrt((dydl*SIGMA_LAMBDA)**2 + (0.5*d2ydl2*SIGMA_LAMBDA**2)) # errore propagato
    popt, pcov = curve_fit(max_fit_parabolic, xrangemax, yrangemax, p0=[λcenter, a, IMAX],sigma=sy)
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))

    λcenter_list.append(λcenter)
    a_list.append(a)
    IMAX_list.append(IMAX)
    error_λcenter_list.append(error_λcenter)
    error_a_list.append(error_a)
    error_IMAX_list.append(error_IMAX)
    sigmay.append(sy)

# to numpy
λcenter_list = np.array(λcenter_list)
a_list = np.array(a_list)
IMAX_list = np.array(IMAX_list)
error_λcenter_list = np.array(error_λcenter_list)
error_a_list = np.array(error_a_list)
error_IMAX_list = np.array(error_IMAX_list)
sigmay = np.array(sigmay)

print('Tabella con i fit di tutti i picchi delle parabole')
max_fit_parabolic_dataframe = pd.DataFrame({'Absorbance': ASS, 'λcenter': λcenter_list, 'a': a_list, 'IMAX': IMAX_list, 'error_λcenter': error_λcenter_list, 'error_a': error_a_list, 'error_IMAX': error_IMAX_list})
display(max_fit_parabolic_dataframe)


Tabella con i fit di tutti i picchi delle parabole


,Absorbance,λcenter,a,IMAX,error_λcenter,error_a,error_IMAX
0,0.480042,554.044219,2.442431,940.693325,0.274804,0.443226,1.342285
1,0.352690,553.742124,2.156915,819.968625,0.285710,0.435038,1.546347
2,0.263436,552.758972,1.744154,718.450202,0.067448,0.081499,0.254055
3,0.174750,551.865053,1.027011,492.753337,0.074358,0.052306,0.178710
4,0.087549,551.494151,0.503627,238.703070,0.147158,0.053501,0.250304
5,0.070475,551.279005,0.386982,170.262114,0.137228,0.036855,0.180003
6,0.052845,551.307117,0.243386,117.772112,0.080942,0.013286,0.070467
7,0.035265,551.090509,0.101588,52.545366,0.128419,0.009120,0.057811
8,0.017633,550.856778,0.031331,19.321189,0.203719,0.004558,0.033509
9,0.008831,550.925065,0.024793,11.330715,0.231624,0.004218,0.036550


In [13]:
center = np.mean(λcenter_list)
sigma = np.std(λcenter_list)
print('λcenter =', final_val(center, sigma, udm='nm'))
print('NOTA: sigmaLambda dichiarata dal costruttore vale 1.5nm e quindi è compatibile con il valore di sigma trovato')

λcenter = 551.94 ± 1.11 nm
NOTA: sigmaLambda dichiarata dal costruttore vale 1.5nm e quindi è compatibile con il valore di sigma trovato


In [14]:
# plotting normalized spectrum (kind of, in the sense that we are not normalizing the area under the curve, but the maximum value of the spectrum)

delta = np.array(λcenter_list) - np.mean(λcenter_list)  # Calculate the average value of λcenter

plt.figure(figsize=(10, 6))

fig = go.Figure()

for λ_col, I_col, c, d in zip(WAVELENGHTS, INTENSITIES,ASS,delta):
    max = data[I_col].max()
    centered_wavelengths = data[λ_col] - d  # Center the wavelengths around the average λcenter
    fig.add_trace(go.Scatter(x=centered_wavelengths,y= data[I_col]*100/ max,
                    mode='lines',
                    name=f'{np.round(c,2)}'))

fig.update_layout(
                  xaxis_title='Wavelength (nm)',
                  yaxis_title='Intensity (%)',
                  height=600,
                  width=800,
                  yaxis=dict(range=[0, 100]),
                  legend=dict(x=0.88, y=1),
                  font=dict(size=14))

fig.write_html(dir_path +r"\html\normalized_centered_emission_spectrum.html")
fig.write_image(dir_path + r"\images\normalized_centered_emission_spectrum.png")
fig.show()

<Figure size 1000x600 with 0 Axes>

The function $ F(C) $ is defined as:

$ F(C, F_0, k, y_0) = F_0 \cdot \left( 1 - e^{-k \cdot C} \right) + y_0$

where:
- $ C $ is the concentration,
- $ F_0 $ is the maximum fluorescence intensity,
- $ k $ is the rate constant,
- $ y_0 $ is the baseline fluorescence intensity.

In [15]:
# fit with the exponential 1-exp(-k*C)

def F_C_fit(C,F0,k,y0):
    return F0*(1-np.exp(-k*C)) + y0

In [16]:
I = np.array(IMAX_list)
sigmaI = np.array(error_IMAX_list) # need to propagate error on lambda d(-ax^2)/dx = -2ax

A = np.array(ASS) # convert to μM
sigmaA = np.array(SIGMA_ASS) # propagate error on C

# iteration 0
popt, pcov = curve_fit(F_C_fit, A, I, p0=[1000, 0.3, 1.0], sigma=sigmaI, maxfev=10000)
F0, k, y0 = popt
error_F0, error_k, error_y0 = np.sqrt(np.diag(pcov))

# iteration 1
dFdC = np.abs(k * F0 * np.exp(-k * A)) # propagate error on C
sigmaTot = np.sqrt(sigmaI**2 + (dFdC * sigmaA)**2) # propagate error on I
popt, pcov = curve_fit(F_C_fit, A, I, p0=[F0, k, y0], sigma=sigmaTot)
F0, k, y0 = popt
error_F0, error_k, error_y0 = np.sqrt(np.diag(pcov))

print('F0:', final_val(F0, error_F0, decimals=0, udm='a.u.'))
print('k:', final_val(k, error_k, decimals=3))
print('y0:', final_val(y0, error_y0, decimals=3, udm='a.u.'))

# Create a scatter plot with error bars using Plotly
fig = go.Figure()

# Add data points with error bars
fig.add_trace(go.Scatter(
    x=A,
    y=I,
    error_y=dict(
        type='data',
        array=sigmaI,
        visible=True
    ),
    error_x=dict(
        type='data',
        array=sigmaA,
        visible=True
    ),
    mode='markers',
    name='Data with error'
))

# Add the fitted curve
a = np.linspace(np.min(A), np.max(A), 1000)
fig.add_trace(go.Scatter(
    x=a,
    y=F_C_fit(a, *popt),
    mode='lines',
    name='Fit'
))

# Update layout for log scale and labels
fig.update_layout(
    xaxis=dict(
        title='Absorbance',
        type='log'
    ),
    yaxis=dict(
        title='Intensity Fluorescence (a.u.)'
    ),
    # title='Intensity Fluorescence vs Concentration',
    height=600,
    width=800,
    legend=dict(x=0, y=1),
    font=dict(size=14)
)

fig.write_html(dir_path +r"\html\FvsAfit.html")
fig.write_image(dir_path + r"\images\FvsAfit.png")
fig.show()

F0: 3437 ± 3543 a.u.
k: 0.813 ± 0.925 
y0: -19.529 ± 7.29 a.u.


In [17]:
# fit lineare del primo set di dati:
limit = 0.06
def F_C_fit_linear(C,k,y0):
    return k*C + y0

I = np.array(IMAX_list)[ASS < limit]
sigmaI = np.array(error_IMAX_list)[ASS < limit] 

A = np.array(ASS)[ASS <limit] 
sigmaA = np.array(SIGMA_ASS)[ASS < limit]

# iteration 0
popt, pcov = curve_fit(F_C_fit_linear, A, I, p0=[1000,1.0],sigma=sigmaI,maxfev=10000)
k, y0 = popt
error_k, error_y0 = np.sqrt(np.diag(pcov))

# iteration 1
dFdC = np.abs(k*F0*np.exp(-k*A)) # propagate error on C
sigmaTot = np.sqrt(sigmaI**2 + (dFdC*sigmaA)**2) # propagate error on I
popt, pcov = curve_fit(F_C_fit_linear, A, I, p0=[k,y0],sigma=sigmaTot)
k, y0 = popt
error_k, error_y0 = np.sqrt(np.diag(pcov))


# print('F0:',final_val(F0,error_F0,decimals=0,udm='a.u.'))
print('k:',final_val(k,error_k,decimals=3,udm='μM^-1'))
print('y0:',final_val(y0,error_y0,decimals=3,udm='a.u.'))

# Create a scatter plot with error bars using Plotly
fig = go.Figure()

# Add data points with error bars
fig.add_trace(go.Scatter(
    x=A,
    y=I,
    error_y=dict(
        type='data',
        array=sigmaI,
        visible=True
    ),
    error_x=dict(
        type='data',
        array=sigmaA,
        visible=True
    ),
    mode='markers',
    name='Data with error'
))

# Add the fitted curve
a = np.linspace(np.min(A), np.max(A), 1000)
fig.add_trace(go.Scatter(
    x=a,
    y=F_C_fit_linear(a, *popt),
    mode='lines',
    name='Fit'
))

# Update layout for log scale and labels
fig.update_layout(
    xaxis=dict(
        title='Concentration (μM)',
        # type='log'
    ),
    yaxis=dict(
        title='Intensity Fluorescence (a.u.)'
    ),
    title='Intensity Fluorescence vs Concentration',
    height=600,
    width=800,
    legend=dict(x=0, y=1),
    font=dict(size=14)
)

fig.write_html(dir_path +r"\html\FvsAfit_linear.html")
fig.write_image(dir_path + r"\images\FvsAfit_lnear.png")
fig.show()


k: 2237.901 ± 421.616 μM^-1
y0: -15.066 ± 10.271 a.u.
